In [22]:
import pandas as pd

In [23]:
df1 = pd.read_csv('./data/location_ref/citynamemulti_councode_cityco.csv')
df1 = df1.drop(columns=['city', 'country'])
df1.head()

,language,cityLabel,alpha2Code,cityCoords
0,nn,Trnava,SK,Point(17.586211111 48.37775)
1,pl,Trnawa,SK,Point(17.586211111 48.37775)
2,ps,ترناوا,SK,Point(17.586211111 48.37775)
3,ro,Trnava,SK,Point(17.586211111 48.37775)
4,rue,Тырнава,SK,Point(17.586211111 48.37775)


In [24]:
df1.shape  # 1122238

(1147454, 4)

In [25]:
df2 = pd.read_csv('./data/location_ref/cityco_citypop.csv')
df2 = df2.drop(columns=['city', 'cityLabel'])
df2 = df2.drop_duplicates() # remove duplicates
df2.head()

,cityCoords,population
0,Point(14.333333 41.066667),75561
1,Point(31.6 4.85),372410
2,Point(39.283333333 -6.8),4715000
3,Point(32.52654 15.60308),5345000
4,Point(10.992777777 45.438611111),252520


In [26]:
df2.shape

(24372, 2)

In [27]:
df1['cityCoords'].nunique()

28046

In [28]:
df_co = pd.DataFrame(df1['cityCoords'].unique())
df_co = df_co.rename(columns = {0: 'cityCoords'})
df_co.shape

(28046, 1)

In [29]:
### remove all the city coordinates that do not appear in the other list
df3 = df2.merge(df_co, how='inner', on='cityCoords',suffixes=(False, False))
#df3 = df3.dropna()
df3.shape

(24351, 2)

In [30]:
ll = df3['cityCoords'].to_list()
len(ll)

24351

In [31]:
inverse_boolean_series = ~(df2.cityCoords).isin(df3.cityCoords)
inverse_filtered_df = df2.cityCoords[inverse_boolean_series]
inverse_filtered_df.shape

(21,)

In [32]:
df6 = df3[df3['cityCoords'].isin(ll)]
df6.shape

(24351, 2)

In [33]:
df1 = df1[df1['cityCoords'].isin(ll)]
df1.shape

(1024614, 4)

In [34]:
df1['cityCoords'].nunique()  # left 28046 right 21859

21859

### Merge to final df

In [35]:
df1['cityCoords'].nunique()  # left 28046 right 21859

21859

In [36]:
df = df1.merge(df6, how='right', on='cityCoords',suffixes=(False, False))
df.shape

(1122238, 5)

In [37]:
df.head()

,language,cityLabel,alpha2Code,cityCoords,population
0,nn,Trnava,SK,Point(17.586211111 48.37775),65382
1,pl,Trnawa,SK,Point(17.586211111 48.37775),65382
2,ps,ترناوا,SK,Point(17.586211111 48.37775),65382
3,ro,Trnava,SK,Point(17.586211111 48.37775),65382
4,rue,Тырнава,SK,Point(17.586211111 48.37775),65382


In [38]:
df.shape

(1122238, 5)

### Final cleanup steps

In [39]:
# remove duplicates cityLabel entries
df_dup_free = df[~df['cityLabel'].duplicated()]

In [40]:
df_dup_free.shape

(328486, 5)

### replace weird letters in popualtion column and cast column to float 


In [42]:
df_dup_free['population'] = df_dup_free['population'].str.replace(r'[a-zA-Z]', '').astype('float')

/Users/lara/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [44]:
# kick out small cities (pop size < 300000)
df_dup_free = df_dup_free[df_dup_free['population'] >= 300000]
df_dup_free.shape

(52345, 5)

### Reorder cordinate cols

In [45]:
# reame columns
df_d = df_dup_free.rename({'alpha2Code': 'COUNTRY_ALPHA2_CODE', 
                                      'cityLabel': 'CITY_NAME', 
                                      'cityCoords': 'city_long_lat_3857'}, axis='columns')

In [46]:
df_d['city_long_lat_3857'] = df_d['city_long_lat_3857'].replace("Point\(", "", regex=True).replace(" ", ", ", regex=True).replace("\)", "", regex=True)

In [47]:
#df_d['city_long_lat_3857'] = df_d['city_long_lat_3857'].apply(lambda x:pd.Series(x.split()[::-1]))


# converting to string series 
df_d['city_long_lat_3857']= df_d['city_long_lat_3857'].astype(str) 
  
# splitting at occurrence of whitespace 
df_d['city_long_lat_3857']= df_d['city_long_lat_3857'].str.split(",", 1) 
  
# displaying second element from list 
df_d['city_lat_3857']  = df_d['city_long_lat_3857'].str.get(1) 

# displaying first element from list 
df_d['city_long_3857']  = df_d['city_long_lat_3857'].str.get(0) 

In [48]:
df_d['city_lat_long_3857'] = df_d['city_lat_3857'] + ", " + df_d['city_long_3857'] 
df_d = df_d.drop(columns='city_long_lat_3857')

In [49]:
df_d.head()

,language,CITY_NAME,COUNTRY_ALPHA2_CODE,population,city_lat_3857,city_long_3857,city_lat_long_3857
81,pl,Nowy Jork,US,8398748.0,40.67,-73.94,"40.67, -73.94"
82,en,New York City,US,8398748.0,40.67,-73.94,"40.67, -73.94"
83,fi,New York,US,8398748.0,40.67,-73.94,"40.67, -73.94"
84,he,ניו יורק,US,8398748.0,40.67,-73.94,"40.67, -73.94"
88,ru,Нью-Йорк,US,8398748.0,40.67,-73.94,"40.67, -73.94"


### Export

In [50]:
df_d.to_csv('./data/location_ref/citynamemulti_councode_cityco_citypop.csv')